<a href="https://colab.research.google.com/github/enyst/AI-playground/blob/main/templates/aistudio_gemini_prompt_freeform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [4]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [8]:
!pip install -U -q "google-generativeai>=0.8.2"

In [10]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = "gemini-1.5-flash"  # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IkV4cGxhaW4gU0dEIGluIHRoZSBNTCBmaWVsZCwgZm9yIGFuIGF1ZGllbmNlIG9mIHNvZnR3YXJlIGRldmVsb3BlcnMgd2hvIGhhdmUgaW50ZXJhY3RlZCB3aXRoIHByZXRyYWluZWQgTExNcywgYnV0IGRvIG5vdCBuZWNlc3NhcmlseSBoYXZlIHRoZSB0ZWNobmljYWwga25vd2xlZGdlIG9mIHRoZSBmdWxsIHRyYWluaW5nIHByb2Nlc3MuIEtlZXAgaXQgc2hvcnQgYW5kIHN3ZWV0LiBEbyBub3QgaW5mYW50aWxpemUgdGhlIGF1ZGllbmNlLlxcbiJ9XX0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6W3sidGV4dCI6IkltYWdpbmUgZmluZS10dW5pbmcgYSBwcmUtdHJhaW5lZCBMTE0uICBZb3UgaGF2ZSBhIGJ1bmNoIG9mIGV4YW1wbGVzIGFuZCB5b3Ugd2FudCB0aGUgbW9kZWwgdG8gcGVyZm9ybSB3ZWxsIG9uIHRoZW0uICBTR0QsIG9yIFN0b2NoYXN0aWMgR3JhZGllbnQgRGVzY2VudCwgaXMgbGlrZSBuYXZpZ2F0aW5nIGEgZm9nZ3kgbW91bnRhaW4uICBZb3Ugd2FudCB0byByZWFjaCB0aGUgbG93ZXN0IHBvaW50IChiZXN0IG1vZGVsIHBlcmZvcm1hbmNlKSwgYnV0IHlvdSBjYW4gb25seSBzZWUgYSBmZXcgZmVldCBhcm91bmQgeW91LlxcblxcbkVhY2ggXFxcInN0ZXBcXFwiIGluIFNHRCBpbnZvbHZlczpcXG5cXG4xLiBQaWNraW5nIGEgc21hbGwsIHJhbmRvbSBiYXRjaCBvZiB5b3VyIGV4YW1wbGVzLlxcbjIuIENhbGN1bGF0aW5nIGhvdyBcXFwid3JvbmdcXFwiIHRoZSBtb2RlbCBpcyBvbiB0aG9zZSBleGFtcGxlcyAodGhlIFxcXCJsb3NzXFxcIikuXFxuMy4gU2xpZ2h0bHkgYWRqdXN0aW5nIHRoZSBtb2RlbCdzIGludGVybmFsIHBhcmFtZXRlcnMgKFxcXCJ3ZWlnaHRzXFxcIikgdG8gaG9wZWZ1bGx5IHJlZHVjZSB0aGUgbG9zcyBvbiBzaW1pbGFyIGV4YW1wbGVzLlxcblxcblRoaXMgcHJvY2VzcyByZXBlYXRzLCBpdGVyYXRpdmVseSBcXFwic3RlcHBpbmdcXFwiIGRvd25oaWxsIHRvd2FyZHMgYmV0dGVyIHBlcmZvcm1hbmNlLiAgXFxcIlN0b2NoYXN0aWNcXFwiIG1lYW5zIHdlJ3JlIHVzaW5nIHNtYWxsIHJhbmRvbSBiYXRjaGVzIGluc3RlYWQgb2YgdGhlIHdob2xlIGRhdGFzZXQgYXQgb25jZSwgd2hpY2ggbWFrZXMgaXQgZmFzdGVyIGFuZCBhbGxvd3MgdGhlIG1vZGVsIHRvIGdlbmVyYWxpemUgYmV0dGVyLiAgXFxcIkdyYWRpZW50XFxcIiByZWZlcnMgdG8gdGhlIGRpcmVjdGlvbiBvZiBzdGVlcGVzdCBkZXNjZW50IGluIHRoZSBsb3NzIGxhbmRzY2FwZS4gIFxcXCJEZXNjZW50XFxcIiBtZWFucyB3ZSdyZSBtb3ZpbmcgdG93YXJkcyBsb3dlciBsb3NzLlxcbiJ9XX1d'  # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9'  # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "Explain SGD in the ML field, for an audience of software developers who have interacted with pretrained LLMs, but do not necessarily have the technical knowledge of the full training process. Keep it short and sweet. Do not infantilize the audience.\\n"
            }
        ]
    },
    {
        "role": "model",
        "parts": [
            {
                "text": "Imagine fine-tuning a pre-trained LLM.  You have a bunch of examples and you want the model to perform well on them.  SGD, or Stochastic Gradient Descent, is like navigating a foggy mountain.  You want to reach the lowest point (best model performance), but you can only see a few feet around you.\\n\\nEach \\\"step\\\" in SGD involves:\\n\\n1. Picking a small, random batch of your examples.\\n2. Calculating how \\\"wrong\\\" the model is on those examples (the \\\"loss\\\").\\n3. Slightly adjusting the model's internal parameters 

## Call `generate_content`

In [11]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

Think of it as a slow, controlled learning process.  SGD helps the LLM learn from its mistakes, gradually improving its performance on your specific examples.  It's the workhorse of training deep learning models, and the foundation of many advancements in AI. 


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [12]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

Explain SGD in the ML field, for an audience of software developers who have interacted with pretrained LLMs, but do not necessarily have the technical knowledge of the full training process. Keep it short and sweet. Do not infantilize the audience.\n 

-------------------------------------------------------------------------------- 

Role: model 

Imagine fine-tuning a pre-trained LLM.  You have a bunch of examples and you want the model to perform well on them.  SGD, or Stochastic Gradient Descent, is like navigating a foggy mountain.  You want to reach the lowest point (best model performance), but you can only see a few feet around you.\n\nEach \"step\" in SGD involves:\n\n1. Picking a small, random batch of your examples.\n2. Calculating how \"wrong\" the model is on those examples (the \"loss\").\n3. Slightly adjusting the model's internal parameters (\"weights\") to hopefully reduce the loss on similar examples.\n\nThis process repeats, iteratively \"stepping\" down